In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import pickle

from gpu_utils import set_n_get_device

%matplotlib inline

In [3]:
import torch
import torch.nn as nn
from dataset import *
#from dataset_single_multi_label import *
#from gpu_utils import set_n_get_device

#device = torch.device("cuda:1")

In [4]:
SEED = 1234
BATCH_SIZE = 4#32#16
NUM_WORKERS = 1#20
debug = True

device = set_n_get_device("0, 1, 2, 3", data_device_id="cuda:3")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
#multi_gpu = [0,1,2]#[1, 2]#None, 0, 1, 2, 3

In [5]:
# train_dl, val_dl = prepare_trainset(BATCH_SIZE, NUM_WORKERS, SEED, debug, 
#                                     sample_mode='weak_balance', use_sampler=False)#balance, weak_balance

#train_dl, val_dl = prepare_trainset(BATCH_SIZE, NUM_WORKERS, SEED, debug)

train_dl, val_dl = prepare_trainset_withMask(BATCH_SIZE, NUM_WORKERS, SEED, debug, 
                                             sample_mode='raw', use_sampler=False)

total trainset:  1000
Count of trainset:  901
Count of validset:  99
Count of trainset (for training):  900
Count of validset (for training):  100
IMG_SIZE:  512
IMG_SIZE:  512


In [6]:
# for i, ( image, (mask, target) ) in enumerate(train_dl):
#     image = image.to(device=device, dtype=torch.float)
#     mask = mask.to(device=device, dtype=torch.float)
#     target = target.to(device=device, dtype=torch.float)
#     if i == 0:
#         break

for i, (image, target) in enumerate(train_dl):
    image = image.to(device=device, dtype=torch.float)
    target = target.to(device=device, dtype=torch.float)
    if i == 0:
        break

In [7]:
image[:, :3, :, :].size(), image[:, 3, :, :].view(-1, 1, 512, 512).size()#[:, :4, :, :]
#image.size(), target.size()

(torch.Size([4, 3, 512, 512]), torch.Size([4, 1, 512, 512]))

In [13]:
# from glob import glob
# import os

# f = glob('data/raw/train/*.png')
# len(f)
# #for _f in f:
# #    os.remove(_f)
# train_dl.dataset.label.loc[train_dl.dataset.label.Id.isin(train_dl.dataset.fname), 
#                            'Target'].apply(lambda x: len(x.split(' '))==1).mean(), \
# val_dl.dataset.label.loc[val_dl.dataset.label.Id.isin(val_dl.dataset.fname), 
#                            'Target'].apply(lambda x: len(x.split(' '))==1).mean()

In [19]:
#plt.imshow(image[1, 1, :, :], cmap='Greens')
#plt.imshow(mask[1, 0, :, :], cmap='binary')

## model

In [8]:
#from model import AtlasResNet18, AtlasResNet34
from model_with_mask import AtlasResNet34
#net = AtlasResNet18(debug=True).cuda(device=device)
net = AtlasResNet34(debug=True).cuda(device=device)

#net = nn.DataParallel(net, device_ids=multi_gpu)

In [14]:
#from model2 import AtlasInceptionV3
#net = AtlasInceptionV3(debug=True, num_classes=28, aux_logits=True, transform_input=False).cuda(device)

#from model3 import AtlasBNInception
#net = AtlasBNInception(debug=True, num_classes=28).cuda(device)

#from model4 import AtlasSEResnet50
#net = AtlasSEResnet50(debug=True).cuda(device=device)

In [8]:
logit_clf, logit_mask = net(image[:, :3, :, :])#[:, :4, :, :]

input:  torch.Size([4, 3, 512, 512])
e1 torch.Size([4, 64, 128, 128])
e2 torch.Size([4, 64, 128, 128])
e3 torch.Size([4, 128, 64, 64])
e4 torch.Size([4, 256, 32, 32])
e5 torch.Size([4, 512, 16, 16])
avgpool torch.Size([4, 512, 1, 1])
reshape torch.Size([4, 512])
clf output torch.Size([4, 28])
center torch.Size([4, 256, 16, 16])
d5 torch.Size([4, 256, 16, 16])
d4 torch.Size([4, 256, 16, 16])
d3 torch.Size([4, 256, 16, 16])
d2 torch.Size([4, 256, 16, 16])
d1 torch.Size([4, 32, 512, 512])
mask output torch.Size([4, 1, 512, 512])


/home/bigdata/data/endi/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/bigdata/data/endi/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [14]:
# 
#output = net(image)#[:, :3, :, :]
logit_clf, logit_mask = net(image[:, :4, :, :])

input:  torch.Size([4, 4, 512, 512])
e1 torch.Size([4, 64, 256, 256])
e2 torch.Size([4, 64, 256, 256])
e3 torch.Size([4, 128, 128, 128])
e4 torch.Size([4, 256, 64, 64])
e5 torch.Size([4, 512, 32, 32])
avgpool torch.Size([4, 512, 1, 1])
reshape torch.Size([4, 512])
clf output torch.Size([4, 28])
center torch.Size([4, 256, 16, 16])
d5 torch.Size([4, 256, 16, 16])
d4 torch.Size([4, 256, 16, 16])
d3 torch.Size([4, 256, 16, 16])
d2 torch.Size([4, 256, 16, 16])
d1 torch.Size([4, 256, 16, 16])
hypercolum torch.Size([4, 320, 512, 512])
mask output torch.Size([4, 1, 512, 512])


/home/bigdata/data/endi/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/bigdata/data/endi/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [9]:
net.criterion(logit_clf, target, logit_mask, mask)

NameError: name 'mask' is not defined

In [16]:
net.metric(logit_clf, target)

/home/bigdata/data/endi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bigdata/data/endi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.055952380952380955

In [9]:
#net.module.criterion(output, target)
net.criterion(output, target)

tensor(5.1288, device='cuda:3', grad_fn=<MeanBackward1>)

In [11]:
%%time
#net.metric(output.cpu().float(), target.cpu().float(), device='cpu')
net.metric(output, target)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 3.35 ms


/home/bigdata/data/endi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bigdata/data/endi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.0976190476190476

In [70]:
output.shape, target.shape

(torch.Size([4, 28]), torch.Size([4, 28]))

In [17]:
torch.cuda.empty_cache()

In [14]:
def macro_f1_torch(logit, truth, threshold=0.5):
    pred = (logit>threshold).float()
    truth = truth.float()
    tp = torch.sum(truth*pred, dim=0)
    tn = torch.sum((1-truth)*(1-pred), dim=0)
    fp = torch.sum((1-truth)*pred, dim=0)
    fn = torch.sum(truth*(1-pred), dim=0)

    p = tp / (tp + fp + 1e-10)
    r = tp / (tp + fn + 1e-10)

    f1 = 2*p*r / (p+r+1e-10)
    f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1), f1)
    return torch.mean(f1)

tensor(0.0102, device='cuda:3')

In [58]:
%%time
macro_f1_torch(output, target)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.02 ms


tensor(0.0102, device='cuda:3')

In [12]:
#layer_name = 'resnet'
#layer_name = '.'

print('Total parameters: ', sum(p[1].numel() for p in net.named_parameters() \
                                if '.' in p[0]))

#print('Trainable parameters: ', sum(p[1].numel() for p in net.named_parameters() \
#                                    if p[1].requires_grad and layer_name in p[0]))

print('Encoder parameters: ', sum(p[1].numel() for p in net.named_parameters() \
                                if 'resnet' in p[0]))

#print('Center parameters: ', sum(p[1].numel() for p in net.named_parameters() \
#                                if 'center' in p[0]))

Total parameters:  21857540
Encoder parameters:  21797672


In [17]:
_net = torchvision.models.inception_v3(pretrained=False)

In [9]:
for k,v in net.named_parameters():
    print(k)

resnet.conv1.weight
resnet.bn1.weight
resnet.bn1.bias
resnet.layer1.0.conv1.weight
resnet.layer1.0.bn1.weight
resnet.layer1.0.bn1.bias
resnet.layer1.0.conv2.weight
resnet.layer1.0.bn2.weight
resnet.layer1.0.bn2.bias
resnet.layer1.1.conv1.weight
resnet.layer1.1.bn1.weight
resnet.layer1.1.bn1.bias
resnet.layer1.1.conv2.weight
resnet.layer1.1.bn2.weight
resnet.layer1.1.bn2.bias
resnet.layer1.2.conv1.weight
resnet.layer1.2.bn1.weight
resnet.layer1.2.bn1.bias
resnet.layer1.2.conv2.weight
resnet.layer1.2.bn2.weight
resnet.layer1.2.bn2.bias
resnet.layer2.0.conv1.weight
resnet.layer2.0.bn1.weight
resnet.layer2.0.bn1.bias
resnet.layer2.0.conv2.weight
resnet.layer2.0.bn2.weight
resnet.layer2.0.bn2.bias
resnet.layer2.0.downsample.0.weight
resnet.layer2.0.downsample.1.weight
resnet.layer2.0.downsample.1.bias
resnet.layer2.1.conv1.weight
resnet.layer2.1.bn1.weight
resnet.layer2.1.bn1.bias
resnet.layer2.1.conv2.weight
resnet.layer2.1.bn2.weight
resnet.layer2.1.bn2.bias
resnet.layer2.2.conv1.weight
r

In [10]:
for param in net.named_parameters():#do not backward ['decoder', 'center'] layers params
    if 'decoder' in param[0]:
        param[1].requires_grad = False
        #print('skipping layer: ', param[0])
    elif 'center' in param[0]:
        param[1].requires_grad = False
        #print('skipping layer: ', param[0])
    elif 'resnet' in param[0] and param[0] not in ['resnet.fc.weight', 'resnet.fc.bias']:#only backward fc layers
        param[1].requires_grad = False
        #print('skipping layer: ', param[0])
    elif 'encoder' in param[0]:
        param[1].requires_grad = False
        #print('skipping layer: ', param[0])
print('='*20)
print('Params that requires_grad=True: ')
for k,v in net.named_parameters():
    if param[1].requires_grad:
        print(k)

Params that requires_grad=True: 
resnet.conv1.weight
resnet.bn1.weight
resnet.bn1.bias
resnet.layer1.0.conv1.weight
resnet.layer1.0.bn1.weight
resnet.layer1.0.bn1.bias
resnet.layer1.0.conv2.weight
resnet.layer1.0.bn2.weight
resnet.layer1.0.bn2.bias
resnet.layer1.1.conv1.weight
resnet.layer1.1.bn1.weight
resnet.layer1.1.bn1.bias
resnet.layer1.1.conv2.weight
resnet.layer1.1.bn2.weight
resnet.layer1.1.bn2.bias
resnet.layer1.2.conv1.weight
resnet.layer1.2.bn1.weight
resnet.layer1.2.bn1.bias
resnet.layer1.2.conv2.weight
resnet.layer1.2.bn2.weight
resnet.layer1.2.bn2.bias
resnet.layer2.0.conv1.weight
resnet.layer2.0.bn1.weight
resnet.layer2.0.bn1.bias
resnet.layer2.0.conv2.weight
resnet.layer2.0.bn2.weight
resnet.layer2.0.bn2.bias
resnet.layer2.0.downsample.0.weight
resnet.layer2.0.downsample.1.weight
resnet.layer2.0.downsample.1.bias
resnet.layer2.1.conv1.weight
resnet.layer2.1.bn1.weight
resnet.layer2.1.bn1.bias
resnet.layer2.1.conv2.weight
resnet.layer2.1.bn2.weight
resnet.layer2.1.bn2.bi

In [2]:
l = [
    'resnet.conv1.weight',
'resnet.bn1.weight',
'resnet.bn1.bias',
'resnet.layer1.0.conv1.weight',
'resnet.layer1.0.bn1.weight',
'resnet.layer1.0.bn1.bias',
'resnet.layer1.0.conv2.weight',
'resnet.layer1.0.bn2.weight',
'resnet.layer1.0.bn2.bias',
'resnet.layer1.1.conv1.weight',
'resnet.layer1.1.bn1.weight',
'resnet.layer1.1.bn1.bias',
'resnet.layer1.1.conv2.weight',
'resnet.layer1.1.bn2.weight',
'resnet.layer1.1.bn2.bias',
'resnet.layer1.2.conv1.weight',
'resnet.layer1.2.bn1.weight',
'resnet.layer1.2.bn1.bias',
'resnet.layer1.2.conv2.weight',
'resnet.layer1.2.bn2.weight',
'resnet.layer1.2.bn2.bias',
'resnet.layer2.0.conv1.weight',
'resnet.layer2.0.bn1.weight',
'resnet.layer2.0.bn1.bias',
'resnet.layer2.0.conv2.weight',
'resnet.layer2.0.bn2.weight',
'resnet.layer2.0.bn2.bias',
'resnet.layer2.0.downsample.0.weight',
'resnet.layer2.0.downsample.1.weight',
'resnet.layer2.0.downsample.1.bias',
'resnet.layer2.1.conv1.weight',
'resnet.layer2.1.bn1.weight',
'resnet.layer2.1.bn1.bias',
'resnet.layer2.1.conv2.weight',
'resnet.layer2.1.bn2.weight',
'resnet.layer2.1.bn2.bias',
'resnet.layer2.2.conv1.weight',
'resnet.layer2.2.bn1.weight',
'resnet.layer2.2.bn1.bias',
'resnet.layer2.2.conv2.weight',
'resnet.layer2.2.bn2.weight',
'resnet.layer2.2.bn2.bias',
'resnet.layer2.3.conv1.weight',
'resnet.layer2.3.bn1.weight',
'resnet.layer2.3.bn1.bias',
'resnet.layer2.3.conv2.weight',
'resnet.layer2.3.bn2.weight',
'resnet.layer2.3.bn2.bias',
'resnet.layer3.0.conv1.weight',
'resnet.layer3.0.bn1.weight',
'resnet.layer3.0.bn1.bias',
'resnet.layer3.0.conv2.weight',
'resnet.layer3.0.bn2.weight',
'resnet.layer3.0.bn2.bias',
'resnet.layer3.0.downsample.0.weight',
'resnet.layer3.0.downsample.1.weight',
'resnet.layer3.0.downsample.1.bias',
'resnet.layer3.1.conv1.weight',
'resnet.layer3.1.bn1.weight',
'resnet.layer3.1.bn1.bias',
'resnet.layer3.1.conv2.weight',
'resnet.layer3.1.bn2.weight',
'resnet.layer3.1.bn2.bias',
'resnet.layer3.2.conv1.weight',
'resnet.layer3.2.bn1.weight',
'resnet.layer3.2.bn1.bias',
'resnet.layer3.2.conv2.weight',
'resnet.layer3.2.bn2.weight',
'resnet.layer3.2.bn2.bias',
'resnet.layer3.3.conv1.weight',
'resnet.layer3.3.bn1.weight',
'resnet.layer3.3.bn1.bias',
'resnet.layer3.3.conv2.weight',
'resnet.layer3.3.bn2.weight',
'resnet.layer3.3.bn2.bias',
'resnet.layer3.4.conv1.weight',
'resnet.layer3.4.bn1.weight',
'resnet.layer3.4.bn1.bias',
'resnet.layer3.4.conv2.weight',
'resnet.layer3.4.bn2.weight',
'resnet.layer3.4.bn2.bias',
'resnet.layer3.5.conv1.weight',
'resnet.layer3.5.bn1.weight',
'resnet.layer3.5.bn1.bias',
'resnet.layer3.5.conv2.weight',
'resnet.layer3.5.bn2.weight',
'resnet.layer3.5.bn2.bias',
'resnet.layer4.0.conv1.weight',
'resnet.layer4.0.bn1.weight',
'resnet.layer4.0.bn1.bias',
'resnet.layer4.0.conv2.weight',
'resnet.layer4.0.bn2.weight',
'resnet.layer4.0.bn2.bias',
'resnet.layer4.0.downsample.0.weight',
'resnet.layer4.0.downsample.1.weight',
'resnet.layer4.0.downsample.1.bias',
'resnet.layer4.1.conv1.weight',
'resnet.layer4.1.bn1.weight',
'resnet.layer4.1.bn1.bias',
'resnet.layer4.1.conv2.weight',
'resnet.layer4.1.bn2.weight',
'resnet.layer4.1.bn2.bias',
'resnet.layer4.2.conv1.weight',
'resnet.layer4.2.bn1.weight',
'resnet.layer4.2.bn1.bias',
'resnet.layer4.2.conv2.weight',
'resnet.layer4.2.bn2.weight',
'resnet.layer4.2.bn2.bias',
'resnet.fc.weight',
'resnet.fc.bias',
'encoder2.1.spatial_gate.linear_1.weight',
'encoder2.1.spatial_gate.linear_1.bias',
'encoder2.1.spatial_gate.linear_2.weight',
'encoder2.1.spatial_gate.linear_2.bias',
'encoder2.1.channel_gate.conv.weight',
'encoder2.1.channel_gate.conv.bias',
'encoder3.1.spatial_gate.linear_1.weight',
'encoder3.1.spatial_gate.linear_1.bias',
'encoder3.1.spatial_gate.linear_2.weight',
'encoder3.1.spatial_gate.linear_2.bias',
'encoder3.1.channel_gate.conv.weight',
'encoder3.1.channel_gate.conv.bias',
'encoder4.1.spatial_gate.linear_1.weight',
'encoder4.1.spatial_gate.linear_1.bias',
'encoder4.1.spatial_gate.linear_2.weight',
'encoder4.1.spatial_gate.linear_2.bias',
'encoder4.1.channel_gate.conv.weight',
'encoder4.1.channel_gate.conv.bias',
'encoder5.1.spatial_gate.linear_1.weight',
'encoder5.1.spatial_gate.linear_1.bias',
'encoder5.1.spatial_gate.linear_2.weight',
'encoder5.1.spatial_gate.linear_2.bias',
'encoder5.1.channel_gate.conv.weight',
'encoder5.1.channel_gate.conv.bias',
'logit_clf.0.weight',
'logit_clf.0.bias',
'logit_clf.2.weight',
'logit_clf.2.bias',
'center.0.conv.weight',
'center.0.bn.weight',
'center.0.bn.bias',
'decoder5.conv1.weight',
'decoder5.conv1.bias',
'decoder5.conv2.weight',
'decoder5.conv2.bias',
'decoder4.conv1.weight',
'decoder4.conv1.bias',
'decoder4.conv2.weight',
'decoder4.conv2.bias',
'decoder3.conv1.weight',
'decoder3.conv1.bias',
'decoder3.conv2.weight',
'decoder3.conv2.bias',
'decoder2.conv1.weight',
'decoder2.conv1.bias',
'decoder2.conv2.weight',
'decoder2.conv2.bias',
'decoder1.conv1.weight',
'decoder1.conv1.bias',
'decoder1.conv2.weight',
'decoder1.conv2.bias',
'logit_mask.0.weight',
'logit_mask.0.bias',
'logit_mask.2.weight',
'logit_mask.2.bias',
'logit_mask.4.weight',
'logit_mask.4.bias'
]

In [3]:
len(l)

167